[View in Colaboratory](https://colab.research.google.com/github/dingaaling/roadExtract/blob/master/cnn.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
import os, cv2, sklearn, random
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from keras.models import load_model

Using TensorFlow backend.


In [0]:
root_dir = os.getcwd() + "/drive/road_extract/data/sample/"
image_dir = root_dir + "train_sample/"
mask_dir = root_dir + "mask_sample/"

imgs_list, mask_imgs_list = [], []


for i, im in enumerate(os.listdir(image_dir)):
  if i < 200:
    imPath = image_dir + im
    maskPath = mask_dir + im.split('_')[0] + "_mask.png"

    imgs_list.append(load_image(imPath))
    mask_imgs_list.append(load_gray_image(maskPath))


imgs = np.asarray(imgs_list)
mask_imgs = np.asarray(mask_imgs_list)

In [0]:
del imgs_list
del mask_imgs_list

In [6]:
print(imgs.shape)
print(mask_imgs.shape)

(200, 1024, 1024, 3)
(200, 1024, 1024)


# Create Array of Windows and Labels

In [0]:
patch_w, patch_h, patch_stride, patch_pad = 16, 16, 16, 0
img_patches = img_crop(imgs[0],patch_w,patch_h,patch_stride,patch_pad)
for i in range(1,imgs.shape[0]):
  img_patches = np.concatenate((img_patches,img_crop(imgs[i],patch_w,patch_h,patch_stride,patch_pad)))
X = img_patches
Y = np.asarray([load_mask(mask_imgs[i],patch_w,patch_h,patch_stride,0) for i in range(mask_imgs.shape[0])])
Y = np.reshape(Y,-1)

In [8]:
print(X.shape)
print(Y.shape)
print(sum(Y))

(793800, 16, 16, 3)
(793800,)
37126


In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [0]:
del img_patches
del imgs
del mask_imgs

In [11]:
X_road = np.zeros((np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_road_2d = np.zeros((np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_nonroad = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),X.shape[1],X.shape[2],X.shape[3]))
Y_nonroad_2d = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),Y_2d.shape[1]))
print(X_road.shape)
print(Y_road_2d.shape)
print(X_nonroad.shape)
print(Y_nonroad_2d.shape)
road_ind = 0
nonroad_ind = 0
for i in range(X.shape[0]):
  if Y[i]>0.5:
    X_road[road_ind,:,:,:] = X[i,:,:,:]
    Y_road_2d[road_ind,:] = Y_2d[i,:]
    road_ind = road_ind+1
  else:
    X_nonroad[nonroad_ind,:,:,:] = X[i,:,:,:]
    Y_nonroad_2d[nonroad_ind,:] = Y_2d[i,:]
    nonroad_ind = nonroad_ind+1

(37126, 16, 16, 3)
(37126, 2)
(756674, 16, 16, 3)
(756674, 2)


In [0]:
X_nonroad_sample = np.zeros(X_road.shape)
Y_2d_nonroad_sample = np.zeros(Y_road_2d.shape)
rand_ind_nonroad = random.sample(np.ndarray.tolist(np.arange(X_nonroad.shape[0])),np.count_nonzero(Y_2d[:,1]))
x_ind = 0
y_ind = 0
for ind in rand_ind_nonroad:
  X_nonroad_sample[x_ind,:,:,:] = X_nonroad[ind,:,:,:]
  Y_2d_nonroad_sample[y_ind,:] = Y_nonroad_2d[ind,:]
  x_ind = x_ind+1
  y_ind = y_ind+1

In [0]:
X_even = np.zeros((2*np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_even_2d = np.zeros((2*np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_even[0:np.count_nonzero(Y_2d[:,1]),:,:,:] = X_road
Y_even_2d[0:np.count_nonzero(Y_2d[:,1]),:] = Y_road_2d
X_even[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:,:,:] = X_nonroad_sample
Y_even_2d[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:] = Y_2d_nonroad_sample

In [0]:
del X, Y, X_road, Y_road_2d, X_nonroad, Y_nonroad_2d, X_nonroad_sample, Y_2d_nonroad_sample

In [15]:
print(X_even.shape)
print(Y_even_2d.shape)

(74252, 16, 16, 3)
(74252, 2)


# CNN Model

In [16]:
pool_size = (2, 2)
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
nb_classes = 2

model = Sequential()

model.add(Convolution2D(64, 5, 5, # 64 5x5 filters
                        border_mode='same',
                        input_shape=(patch_w,patch_h, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, # 128 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, W_regularizer=l2(reg)))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(16, 16, 3..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/us

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [0]:
adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(X_even,Y_even_2d, batch_size=128, epochs=200)

Epoch 1/200
74252/74252 [==============================] - 20s 274us/step - loss: 3.7196 - acc: 0.5273
Epoch 2/200
74252/74252 [==============================] - 17s 224us/step - loss: 1.0973 - acc: 0.5600
Epoch 3/200
11904/74252 [===>..........................] - ETA: 13s - loss: 0.8863 - acc: 0.5846

74252/74252 [==============================] - 17s 223us/step - loss: 0.8386 - acc: 0.5883
Epoch 4/200
74252/74252 [==============================] - 16s 222us/step - loss: 0.7500 - acc: 0.6078
Epoch 5/200
21888/74252 [=======>......................] - ETA: 11s - loss: 0.7129 - acc: 0.6225

74252/74252 [==============================] - 17s 223us/step - loss: 0.7030 - acc: 0.6256
Epoch 6/200
74252/74252 [==============================] - 17s 223us/step - loss: 0.6721 - acc: 0.6409
Epoch 7/200
24960/74252 [=========>....................] - ETA: 10s - loss: 0.6510 - acc: 0.6542

74252/74252 [==============================] - 17s 223us/step - loss: 0.6458 - acc: 0.6565
Epoch 8/200
74252/74252 [==============================] - 17s 223us/step - loss: 0.6350 - acc: 0.6634
Epoch 9/200
25984/74252 [=========>....................] - ETA: 10s - loss: 0.6252 - acc: 0.6691

74252/74252 [==============================] - 16s 222us/step - loss: 0.6236 - acc: 0.6725
Epoch 10/200
74252/74252 [==============================] - 17s 222us/step - loss: 0.6132 - acc: 0.6815
Epoch 11/200
25728/74252 [=========>....................] - ETA: 10s - loss: 0.6110 - acc: 0.6841

...

74252/74252 [==============================] - 18s 237us/step - loss: 0.4263 - acc: 0.8023
Epoch 192/200
74252/74252 [==============================] - 17s 235us/step - loss: 0.4247 - acc: 0.8025
Epoch 193/200
24960/74252 [=========>....................] - ETA: 11s - loss: 0.4249 - acc: 0.8043

74252/74252 [==============================] - 18s 238us/step - loss: 0.4247 - acc: 0.8032
Epoch 194/200
74252/74252 [==============================] - 18s 236us/step - loss: 0.4222 - acc: 0.8034
Epoch 195/200
24704/74252 [========>.....................] - ETA: 11s - loss: 0.4216 - acc: 0.8052

74252/74252 [==============================] - 18s 237us/step - loss: 0.4234 - acc: 0.8030
Epoch 196/200
74252/74252 [==============================] - 17s 235us/step - loss: 0.4230 - acc: 0.8025
Epoch 197/200
24704/74252 [========>.....................] - ETA: 11s - loss: 0.4263 - acc: 0.8005

74252/74252 [==============================] - 18s 238us/step - loss: 0.4234 - acc: 0.8033
Epoch 198/200
74252/74252 [==============================] - 18s 236us/step - loss: 0.4223 - acc: 0.8034
Epoch 199/200
24704/74252 [========>.....................] - ETA: 11s - loss: 0.4256 - acc: 0.8010

74252/74252 [==============================] - 18s 237us/step - loss: 0.4220 - acc: 0.8052
Epoch 200/200
74252/74252 [==============================] - 18s 236us/step - loss: 0.4213 - acc: 0.8044


In [0]:
model.save("model.h5")
model.save_weights("model_weights.h5")

# Testing

In [0]:
from keras.models import load_model
model = load_model('model.h5')

In [0]:
root_dir = os.getcwd() + "/drive/road_extract/data/sample/"
image_dir = root_dir + "test_sample/"
mask_dir = root_dir + "test_sample_mask/"

imgs_list, mask_imgs_list = [], []


for i, im in enumerate(os.listdir(image_dir)):
  if i < 50:
    imPath = image_dir + im
    maskPath = mask_dir + im.split('_')[0] + "_mask.png"

    imgs_list.append(load_image(imPath))
    mask_imgs_list.append(load_gray_image(maskPath))


imgs = np.asarray(imgs_list)
mask_imgs = np.asarray(mask_imgs_list)

In [0]:
del imgs_list
del mask_imgs_list

In [9]:
print(imgs.shape)
print(mask_imgs.shape)

(50, 1024, 1024, 3)
(50, 1024, 1024)


In [0]:
patch_w, patch_h, patch_stride, patch_pad = 16, 16, 16, 0
img_patches = img_crop(imgs[0],patch_w,patch_h,patch_stride,patch_pad)
for i in range(1,imgs.shape[0]):
  img_patches = np.concatenate((img_patches,img_crop(imgs[i],patch_w,patch_h,patch_stride,patch_pad)))
X = img_patches
Y = np.asarray([load_mask(mask_imgs[i],patch_w,patch_h,patch_stride,0) for i in range(mask_imgs.shape[0])])
Y = np.reshape(Y,-1)

In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [12]:
print(X.shape)
print(Y.shape)
print(sum(Y))
print(Y_2d.shape)


del img_patches
del imgs
del mask_imgs

(198450, 16, 16, 3)
(198450,)
7995
(198450, 2)


In [0]:
output = model.predict(X)

In [15]:
print(output.shape)
print(Y_2d.shape)

(198450, 2)
(198450, 2)


In [16]:
correct, falsePos, falseNeg = 0, 0, 0

for i, scores in enumerate(output):

    pred = np.argmax(scores).astype( np.float32 )
    label = np.argmax(Y_2d[i]).astype( np.float32 )
#     print(i, scores, pred, label)
    if(pred == label):
        correct+=1
    else:
        if(pred==1):
            falsePos+=1
        elif(pred==0):
            falseNeg+=1
print("Prediction Scores")
print("Accuracy: ", (correct/len(output)))
print("False Positive: ",falsePos, "False Negative: ",falseNeg)

Prediction Scores
Accuracy:  0.8105618543713782
False Positive:  36087 False Negative:  1507


In [13]:
score = model.evaluate(X, Y_2d, batch_size=128)
print(score)

198450/198450 [==============================] - 14s 70us/step
[0.37368449107450724, 0.8105618543707774]


# Helper Functions

In [0]:
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    num_patches = int((im.shape[0]-w)*(im.shape[1]-h)/(stride*stride))
    np_patches = np.ndarray((num_patches,w,h,3))
    imgwidth = im.shape[0]
    half_w = int(w/2)
    half_h = int(h/2)
    imgheight = im.shape[1]
#     im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    ind = 0
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
        for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
            im_patch = im[j-half_w-padding:j+half_w+padding, i-half_h-padding:i+half_h+padding, :]
            np_patches[ind,:,:,:] = im_patch
            ind = ind+1
    return np_patches
      
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
  
def load_mask(img,w,h,stride,padding):
    list_patches = []
    imgwidth = img.shape[0]
    imgheight = img.shape[1]
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
      for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
        if img[j,i]>0.5:
          list_patches.append(1)
        else:
          list_patches.append(0)
    return list_patches

In [0]:
def load_image(fileName):
    return mpimg.imread(fileName)
  
def load_gray_image(infilename):
    img = cv2.imread(infilename)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray, (72,72)) 
    return gray
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
      
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X